Processing
---

This notebook prepares the data for modeling. Null values in numeric data are imputed. Lists of car features are cleaned and consolidated (for example, colors `black` and `Black` are considered the same). Strings and lists of string features are split, and the most common are made into dummies.

In [13]:
#import libraries
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# import sklearn

#display options
pd.options.display.max_columns = 40
%matplotlib inline
plt.style.use('dark_background')

In [11]:
#import data
data_path = '../data/'
train_data_filename = 'Training_DataSet.csv'
test_data_filename = 'Test_Dataset.csv'

traindf = pd.read_csv(data_path + train_data_filename)


#copy training data to a new dataframe to use for modeling
traindf_proc = traindf.copy()
traindf_proc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6298 entries, 0 to 6297
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ListingID             6298 non-null   int64  
 1   SellerCity            6298 non-null   object 
 2   SellerIsPriv          6298 non-null   bool   
 3   SellerListSrc         6296 non-null   object 
 4   SellerName            6298 non-null   object 
 5   SellerRating          6298 non-null   float64
 6   SellerRevCnt          6298 non-null   int64  
 7   SellerState           6298 non-null   object 
 8   SellerZip             6296 non-null   float64
 9   VehBodystyle          6298 non-null   object 
 10  VehCertified          6298 non-null   bool   
 11  VehColorExt           6225 non-null   object 
 12  VehColorInt           5570 non-null   object 
 13  VehDriveTrain         5897 non-null   object 
 14  VehEngine             5937 non-null   object 
 15  VehFeats             

---
# Processing feature by feature

# ListingID

drop it.

In [83]:
#keep track to drop later.
columns_to_drop = ['ListingID']



## SellerCity
Although SellerCity did not appear to influence the average dealer listing price, it is possible that patterns with SellerCity could be paired with other features to derive the value. This will not be effective in a linear regression model but could be grabbed onto in a decision tree / NN / etc.

In [36]:
#make dummies for Seller Cities with the most sales in the training set.
#perhaps a bit arbitrary, but let's cut it off at cities above 30 sales.
#that's the first 20 common cities.
cities = traindf['SellerCity'].value_counts(ascending= False)[:20]
cities = cities.index
cities

Index(['Chicago', 'Battle Creek', 'Columbus', 'Louisville', 'Houston',
       'Atlanta', 'Richmond', 'Raleigh', 'Indianapolis', 'Vienna',
       'Cincinnati', 'Dallas', 'White Bear Lake', 'Palmyra', 'Rochester',
       'Nashville', 'Milwaukee', 'St. Louis', 'Lexington', 'Pittsburgh'],
      dtype='object')

In [46]:
# takes a dataframe, a column to expand upon, and a list of values for that column to make dummies.
# any value in the column that is not in the dummy_list will be ignored

#the purpose of this is to make the same ordered columns in the training set as in any test set. Otherwise,
#the dummy columns may notinclude the same cities, if there is a different frequency distribution,
#or if a certain city isn't represented.
def make_specific_dummies(df, column, dummy_list):
    #make a copy so we don't change the original
    df2 = df.copy()
    #remove any entries in column that aren't in the dummy list
    for dummy_value in dummy_list:
        df2[column + '_' + dummy_value] = df2[column].apply(lambda entry: 1 if entry == dummy_value else 0)
    return df2.drop(columns = column)
    

In [48]:
traindf = make_specific_dummies(traindf, 'SellerCity', cities)
traindf.head()

,ListingID,SellerIsPriv,SellerListSrc,SellerName,SellerRating,SellerRevCnt,SellerState,SellerZip,VehBodystyle,VehCertified,VehColorExt,VehColorInt,VehDriveTrain,VehEngine,VehFeats,VehFuel,VehHistory,VehListdays,VehMake,VehMileage,...,SellerCity_Chicago,SellerCity_Battle Creek,SellerCity_Columbus,SellerCity_Louisville,SellerCity_Houston,SellerCity_Atlanta,SellerCity_Richmond,SellerCity_Raleigh,SellerCity_Indianapolis,SellerCity_Vienna,SellerCity_Cincinnati,SellerCity_Dallas,SellerCity_White Bear Lake,SellerCity_Palmyra,SellerCity_Rochester,SellerCity_Nashville,SellerCity_Milwaukee,SellerCity_St. Louis,SellerCity_Lexington,SellerCity_Pittsburgh
0,3287,False,Inventory Command Center,Prime Motorz,5.0,32,MI,48091.0,SUV,False,White,Black,4X4,3.6L V6,"['Adaptive Cruise Control', 'Antilock Brakes',...",Gasoline,"1 Owner, Non-Personal Use Reported, Buyback Pr...",8.600069,Jeep,39319.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,3920,False,Cadillac Certified Program,Gateway Chevrolet Cadillac,4.8,1456,ND,58103.0,SUV,True,Black,NaN,NaN,NaN,NaN,Gasoline,"1 Owner, Buyback Protection Eligible",2.920127,Cadillac,30352.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,4777,False,Jeep Certified Program,Wilde Chrysler Jeep Dodge Ram &amp; Subaru,4.8,1405,WI,53186.0,SUV,True,Brilliant Black Crystal Pearlcoat,Black,4x4/4WD,Regular Unleaded V-6 3.6 L/220,['18 WHEEL &amp; 8.4 RADIO GROUP-inc: Nav-Capa...,Gasoline,"1 Owner, Buyback Protection Eligible",28.107014,Jeep,38957.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,6242,False,Inventory Command Center,Century Dodge Chrysler Jeep RAM,4.4,21,MO,63385.0,SUV,False,Diamond Black Crystal Pearlcoat,Black,4WD,3.6L V6,"['Android Auto', 'Antilock Brakes', 'Apple Car...",Gasoline,"1 Owner, Non-Personal Use Reported, Buyback Pr...",59.816875,Jeep,20404.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,7108,False,HomeNet Automotive,Superior Buick GMC of Fayetteville,3.7,74,AR,72703.0,SUV,False,Radiant Silver Metallic,Cirrus,FWD,Gas V6 3.6L/222.6,"['4-Wheel Disc Brakes', 'ABS', 'Adjustable Ste...",Gasoline,"1 Owner, Non-Personal Use Reported, Buyback Pr...",98.665301,Cadillac,19788.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# SellerListSrc

Only two are nulls --- this will leave those as the only 0 for dummies. (no dropfirst)
Again, `pd.get_dummies` could be a problem if there are a different order. Make dummy columns manually.

In [59]:
sources = traindf['SellerListSrc'].dropna().unique()
sources

array(['Inventory Command Center', 'Cadillac Certified Program',
       'Jeep Certified Program', 'HomeNet Automotive',
       'Digital Motorworks (DMi)', 'My Dealer Center', 'Sell It Yourself',
       'Five Star Certified Program'], dtype=object)

In [61]:
traindf = make_specific_dummies(traindf, 'SellerListSrc', sources)

## SellerName
Treat this like cities, with the most popular names marked. Again, arbitrarily for now, just picking the top 20 sellers.
Some sellers might be tied with low prices for certain models.

In [67]:
#not taking unique because there are so many. order by value count first, then take the first collection
sellers = traindf['SellerName'].value_counts(ascending = False)[:20]
sellers = sellers.index
sellers

Index(['Vroom (Online Dealer - Nationwide Delivery)', 'Carvana',
       'Henkel Chrysler Dodge Jeep Ram',
       'OffLeaseOnly.com The Nation&#x27;s Used Car Destination',
       'Koons Tysons Chrysler Dodge Jeep RAM',
       'Marino Chrysler Jeep Dodge RAM', 'Barnett Chrysler Jeep Kia',
       'F.C. Kerbeck &amp; Sons', 'Sewell Cadillac of Dallas',
       'Blue Knob Auto Sales', 'CarMax White Marsh',
       'Larry Roesch Dodge Chrysler Jeep RAM', 'Cross Chrysler Jeep Fiat',
       'Sewell Cadillac', 'Park Chrysler Jeep', 'Tom Masano Ford Lincoln',
       'Brad Deery Motors', 'Sherman Dodge Chrysler Jeep RAM',
       'Germain Cadillac of Easton', 'Westgate Chrysler Jeep Dodge RAM'],
      dtype='object')

In [68]:
traindf = make_specific_dummies(traindf, 'SellerName', sellers)

## SellerState
All 50 states are represented. The limit of US states means that we can drop one. Let's make it HI just because it has the fewest sales in this training set.

In [78]:
states = traindf['SellerState'].value_counts(ascending = False).index[:-1]
states

Index(['IL', 'OH', 'MI', 'WI', 'PA', 'IN', 'TX', 'NY', 'VA', 'MD', 'FL', 'NC',
       'MO', 'GA', 'TN', 'KY', 'MN', 'IA', 'CA', 'NJ', 'SC', 'CO', 'WV', 'OK',
       'WA', 'LA', 'AZ', 'AL', 'MA', 'AR', 'NV', 'ID', 'UT', 'MS', 'CT', 'NE',
       'SD', 'NH', 'OR', 'KS', 'DE', 'NM', 'VT', 'ND', 'AK', 'RI', 'WY', 'ME',
       'MT'],
      dtype='object')

In [81]:
traindf = make_specific_dummies(traindf,'SellerState', states)

# SellerZip, VehBodystyle
drop.

In [84]:
columns_to_drop.append('SellerZip', 'VehBodystyle')

# VehColorExt

colors to consolidate:
   
   - gray, platinum, silver, steel, granite, billet, billiet, Gy, sil, rhino
   - black, Midnight Sky, Shadow, charcoal
   - white, ivory
   - blue
   - brown, brownstone, mocha
   - beige, beigh, cashmere, bronze, tan
   - gold
   - purple, amethyst, velvet
   - deep red, dark red, deep cherry red, burgundy, sangria, deep auburn, maroon, 
   - red, red horizon
   - (black forest) green
   - pink
   
textures / coats:

    - metallic, me
    - pearl, pearlcoat, pearl-coat
    - crystal
    - clear, clearcoat
    - 3-coat, tricoat, tri-coat
    - tintcoat
    
nonvalues:

    - nan
    - unspecified
    - Not Specified

In [86]:
colors = ['silver', 'black','white', 'blue', 'brown', 'tan', 'gold', 'purple', 'deep red', 'red', 'green', 'pink']

textures = ['metallic', 'pearl', 'crystal', 'clearcoat', 'tintcoat', 'tricoat']

silvers = ['gray', 'platinum', 'silver', 'steel', 'granite', 'billet', 'billiet', 'Gy', 'sil', 'rhino']
blacks = ['black', 'midnight','shadow','charcoal']
whites = ['white','ivory']
browns = ['brown','brownstone','mocha']
tans = ['beige','beigh','cashmere','brown','tan']
purples = ['purple','amethyst','velvet']
deepreds = ['deep red','dark red','deep cherry red','burgundy','sangria','deep auburn','maroon']


In [89]:
traindf.drop(columns = ['fart','doublefart'])

,ListingID,SellerIsPriv,SellerRating,SellerRevCnt,SellerZip,VehBodystyle,VehCertified,VehColorExt,VehColorInt,VehDriveTrain,VehEngine,VehFeats,VehFuel,VehHistory,VehListdays,VehMake,VehMileage,VehModel,VehPriceLabel,VehSellerNotes,...,SellerState_AR,SellerState_NV,SellerState_ID,SellerState_UT,SellerState_MS,SellerState_CT,SellerState_NE,SellerState_SD,SellerState_NH,SellerState_OR,SellerState_KS,SellerState_DE,SellerState_NM,SellerState_VT,SellerState_ND,SellerState_AK,SellerState_RI,SellerState_WY,SellerState_ME,SellerState_MT
0,3287,False,5.0,32,48091.0,SUV,False,White,Black,4X4,3.6L V6,"['Adaptive Cruise Control', 'Antilock Brakes',...",Gasoline,"1 Owner, Non-Personal Use Reported, Buyback Pr...",8.600069,Jeep,39319.0,Grand Cherokee,Fair Price,NaN,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,3920,False,4.8,1456,58103.0,SUV,True,Black,NaN,NaN,NaN,NaN,Gasoline,"1 Owner, Buyback Protection Eligible",2.920127,Cadillac,30352.0,XT5,Good Deal,Come take a look at our great pre-owned invent...,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,4777,False,4.8,1405,53186.0,SUV,True,Brilliant Black Crystal Pearlcoat,Black,4x4/4WD,Regular Unleaded V-6 3.6 L/220,['18 WHEEL &amp; 8.4 RADIO GROUP-inc: Nav-Capa...,Gasoline,"1 Owner, Buyback Protection Eligible",28.107014,Jeep,38957.0,Grand Cherokee,Good Deal,Backed by a rigorous 125-point inspection by f...,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,6242,False,4.4,21,63385.0,SUV,False,Diamond Black Crystal Pearlcoat,Black,4WD,3.6L V6,"['Android Auto', 'Antilock Brakes', 'Apple Car...",Gasoline,"1 Owner, Non-Personal Use Reported, Buyback Pr...",59.816875,Jeep,20404.0,Grand Cherokee,Good Deal,Drop by to see us and you will quickly see how...,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,7108,False,3.7,74,72703.0,SUV,False,Radiant Silver Metallic,Cirrus,FWD,Gas V6 3.6L/222.6,"['4-Wheel Disc Brakes', 'ABS', 'Adjustable Ste...",Gasoline,"1 Owner, Non-Personal Use Reported, Buyback Pr...",98.665301,Cadillac,19788.0,XT5,Good Deal,"Luxury, Exterior Parking Camera Rear, Front Du...",...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6293,8615125,True,0.0,0,48126.0,SUV,False,Black,Black,4x4/4-wheel drive,6-cylinder,NaN,Gasoline,NaN,29.781968,Jeep,49000.0,Grand Cherokee,NaN,****ALL BLACK EDITION****You are viewing a bea...,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6294,8615510,False,3.3,16,46280.0,SUV,False,Gray,Black,4WD,3.0L V6 24V DDI DOHC Turbo Diesel,"['1st and 2nd row curtain head airbags', '4-wh...",Diesel,"1 Owner, Accident(s) Reported, Non-Personal Us...",4.840069,Jeep,20039.0,Grand Cherokee,Fair Price,CarMax makes car buying easy and hassle-free. ...,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6295,8616294,False,4.1,20,43017.0,SUV,True,Black,Jet Black,FWD,3.6L V6 24V GDI DOHC,"['1st and 2nd row curtain head airbags', '4-wh...",Gasoline,"0 Owners, Buyback Protection Eligible",184.921991,Cadillac,16278.0,XT5,Good Deal,Clean CARFAX. Certified. Black 2018 Cadillac X...,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6296,8617378,False,4.9,278,44870.0,SUV,False,Black,Jet Black,FWD,3.6L V6 24V GDI DOHC,"['1st and 2nd row curtain head airbags', '4-wh...",Gasoline,"2 Owners, Non-Personal Use Reported, Buyback P...",73.868426,Cadillac,38146.0,XT5,Great Deal,Black 2017 Cadillac XT5 Luxury FWD 8-Speed Aut...,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [88]:
traindf

,ListingID,SellerIsPriv,SellerRating,SellerRevCnt,SellerZip,VehBodystyle,VehCertified,VehColorExt,VehColorInt,VehDriveTrain,VehEngine,VehFeats,VehFuel,VehHistory,VehListdays,VehMake,VehMileage,VehModel,VehPriceLabel,VehSellerNotes,...,SellerState_ID,SellerState_UT,SellerState_MS,SellerState_CT,SellerState_NE,SellerState_SD,SellerState_NH,SellerState_OR,SellerState_KS,SellerState_DE,SellerState_NM,SellerState_VT,SellerState_ND,SellerState_AK,SellerState_RI,SellerState_WY,SellerState_ME,SellerState_MT,fart,doublefart
0,3287,False,5.0,32,48091.0,SUV,False,White,Black,4X4,3.6L V6,"['Adaptive Cruise Control', 'Antilock Brakes',...",Gasoline,"1 Owner, Non-Personal Use Reported, Buyback Pr...",8.600069,Jeep,39319.0,Grand Cherokee,Fair Price,NaN,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,3920,False,4.8,1456,58103.0,SUV,True,Black,NaN,NaN,NaN,NaN,Gasoline,"1 Owner, Buyback Protection Eligible",2.920127,Cadillac,30352.0,XT5,Good Deal,Come take a look at our great pre-owned invent...,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,4777,False,4.8,1405,53186.0,SUV,True,Brilliant Black Crystal Pearlcoat,Black,4x4/4WD,Regular Unleaded V-6 3.6 L/220,['18 WHEEL &amp; 8.4 RADIO GROUP-inc: Nav-Capa...,Gasoline,"1 Owner, Buyback Protection Eligible",28.107014,Jeep,38957.0,Grand Cherokee,Good Deal,Backed by a rigorous 125-point inspection by f...,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,6242,False,4.4,21,63385.0,SUV,False,Diamond Black Crystal Pearlcoat,Black,4WD,3.6L V6,"['Android Auto', 'Antilock Brakes', 'Apple Car...",Gasoline,"1 Owner, Non-Personal Use Reported, Buyback Pr...",59.816875,Jeep,20404.0,Grand Cherokee,Good Deal,Drop by to see us and you will quickly see how...,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,7108,False,3.7,74,72703.0,SUV,False,Radiant Silver Metallic,Cirrus,FWD,Gas V6 3.6L/222.6,"['4-Wheel Disc Brakes', 'ABS', 'Adjustable Ste...",Gasoline,"1 Owner, Non-Personal Use Reported, Buyback Pr...",98.665301,Cadillac,19788.0,XT5,Good Deal,"Luxury, Exterior Parking Camera Rear, Front Du...",...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6293,8615125,True,0.0,0,48126.0,SUV,False,Black,Black,4x4/4-wheel drive,6-cylinder,NaN,Gasoline,NaN,29.781968,Jeep,49000.0,Grand Cherokee,NaN,****ALL BLACK EDITION****You are viewing a bea...,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6294,8615510,False,3.3,16,46280.0,SUV,False,Gray,Black,4WD,3.0L V6 24V DDI DOHC Turbo Diesel,"['1st and 2nd row curtain head airbags', '4-wh...",Diesel,"1 Owner, Accident(s) Reported, Non-Personal Us...",4.840069,Jeep,20039.0,Grand Cherokee,Fair Price,CarMax makes car buying easy and hassle-free. ...,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6295,8616294,False,4.1,20,43017.0,SUV,True,Black,Jet Black,FWD,3.6L V6 24V GDI DOHC,"['1st and 2nd row curtain head airbags', '4-wh...",Gasoline,"0 Owners, Buyback Protection Eligible",184.921991,Cadillac,16278.0,XT5,Good Deal,Clean CARFAX. Certified. Black 2018 Cadillac X...,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6296,8617378,False,4.9,278,44870.0,SUV,False,Black,Jet Black,FWD,3.6L V6 24V GDI DOHC,"['1st and 2nd row curtain head airbags', '4-wh...",Gasoline,"2 Owners, Non-Personal Use Reported, Buyback P...",73.868426,Cadillac,38146.0,XT5,Great Deal,Black 2017 Cadillac XT5 Luxury FWD 8-Speed Aut...,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [85]:
traindf['VehColorExt'].uni

0                                   White
1                                   Black
2       Brilliant Black Crystal Pearlcoat
3         Diamond Black Crystal Pearlcoat
4                 Radiant Silver Metallic
                      ...                
6293                                Black
6294                                 Gray
6295                                Black
6296                                Black
6297      Diamond Black Crystal Pearlcoat
Name: VehColorExt, Length: 6298, dtype: object

Columns to dummify, and in this order:

- SellerCity: cities
- SellerName: sellers
- SellerState: states

In [71]:

traindf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6298 entries, 0 to 6297
Data columns (total 74 columns):
 #   Column                                                              Non-Null Count  Dtype  
---  ------                                                              --------------  -----  
 0   ListingID                                                           6298 non-null   int64  
 1   SellerIsPriv                                                        6298 non-null   bool   
 2   SellerRating                                                        6298 non-null   float64
 3   SellerRevCnt                                                        6298 non-null   int64  
 4   SellerState                                                         6298 non-null   object 
 5   SellerZip                                                           6296 non-null   float64
 6   VehBodystyle                                                        6298 non-null   object 
 7   VehCertified   